In [34]:
import pandas as pd
import glob
import numpy as np 

def get_d_types(df: pd.DataFrame):
    return [(str(c), str(df[c].dtype)) for c in df.columns]

dfs = [pd.read_parquet(path) for path in glob.glob("./cv/*/train.parquet")]

def get_nominal_indexes(df: pd.DataFrame) -> list[int]:
    """Return indices of nominal columns in given dataframe

    Args:
        df (pd.DataFrame): DataFrame

    Returns:
        list[int]: list of indices of nominal columns
    """
    dtype_mask = df.dtypes == "object"
    nominal_indexes = np.where(dtype_mask)[0]
    return nominal_indexes.tolist()


def get_numerical_indexes(df: pd.DataFrame) -> list[int]:
    """Return indices of numerical columns in given dataframe

    Args:
        df (pd.DataFrame): DataFrame

    Returns:
        list[int]: list of indices of numerical columns
    """
    dtype_mask = np.logical_not(df.dtypes == "object")
    numerical_indexes = np.where(dtype_mask)[0]
    return numerical_indexes.tolist()


In [35]:
df

,survival_time,survival_status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
0,400.0,1.0,1.0,58.765229,f,1.0,1.0,1.0,1.0,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,4500.0,0.0,1.0,56.446270,f,0.0,1.0,1.0,0.0,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,1012.0,1.0,1.0,70.072553,m,0.0,0.0,0.0,0.5,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,1925.0,1.0,1.0,54.740589,f,0.0,1.0,1.0,0.5,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,1504.0,0.0,2.0,38.105407,f,0.0,1.0,1.0,0.0,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,681.0,1.0,NaN,67.000684,f,NaN,NaN,NaN,0.0,1.2,NaN,2.96,NaN,NaN,NaN,NaN,174.0,10.9,3.0
372,1103.0,0.0,NaN,39.000684,f,NaN,NaN,NaN,0.0,0.9,NaN,3.83,NaN,NaN,NaN,NaN,180.0,11.2,4.0
373,1055.0,0.0,NaN,56.999316,f,NaN,NaN,NaN,0.0,1.6,NaN,3.42,NaN,NaN,NaN,NaN,143.0,9.9,3.0
374,691.0,0.0,NaN,58.001369,f,NaN,NaN,NaN,0.0,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269.0,10.4,3.0


In [36]:
df =dfs[0]


a = set(get_nominal_indexes(df)).union(set(get_numerical_indexes(df)))
a

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

In [37]:
set(list(range(df.shape[1])))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}

In [38]:
from lwrules.conditions_induction import ConditionsGenerator

def split_X_y_and_normalize(df: pd.DataFrame):
    X, y = df.drop('survival_status', axis=1), df['survival_status']
    # decision_rules requires data sorted by time
    X = X.sort_values(by='survival_time', ascending=True)
    # decision_rules requires "0" "1" string values
    y = y.astype(int).astype(str)
    return X, y


X, y = split_X_y_and_normalize(df)
ConditionsGenerator(X.to_numpy(), y.to_numpy(), 
                    get_numerical_indexes(df) )

TypeError: ConditionsGenerator.__init__() missing 1 required positional argument: 'nominal_attributes_indices'

In [24]:
import sys
sys.path.append('../../../')